In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
%matplotlib inline

In [ ]:
def lstm_model1(path):
     df = pd.read_csv(path)
     df1 = df.reset_index()['Close']
     from sklearn.preprocessing import MinMaxScaler
     scaler = MinMaxScaler(feature_range = (0,1))
     df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))
     training_size =  2400
     test_size = 600
     train_data , test_data = df1[0:training_size,:],df1[training_size :, :1]
     def create_dataset(dataset , time_step = 250):
       dataX , dataY = [] , []
       for j in range(len(dataset)-time_step-1):
         a= dataset[j:(j+time_step),0]
         dataX.append(a)
         dataY.append(dataset[j + time_step , 0])
       return np.array(dataX), np.array(dataY)
     time_step = 250
     X_train , y_train = create_dataset(train_data , time_step)
     X_test , y_test = create_dataset(test_data , time_step)
     X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
     X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
     import tensorflow as tf
     from tensorflow.keras.models import Sequential
     from tensorflow.keras.layers import Dense
     from tensorflow.keras.layers import LSTM
     model = Sequential()
     model.add(LSTM(50,return_sequences = True,input_shape=(250,1)))
     model.add(LSTM(50,return_sequences=True))
     model.add(LSTM(50))
     model.add(Dense(1))
     model.compile(loss='mean_squared_error',optimizer = 'adam')
     model.fit(X_train,y_train , validation_data=(X_test,y_test),epochs = 100 , batch_size=64,verbose = 1)
     train_predict = model.predict(X_train)
     test_predict = model.predict(X_test)
     train_predict = scaler.inverse_transform(train_predict)
     test_predict = scaler.inverse_transform(test_predict)
     x_input = test_data[350:].reshape(1,-1)
     temp_input = list(x_input)
     lst_output = []
     n_steps = 250
     k=0
     while(k<100):
       if(len(temp_input)>250):
         x_input = np.array(temp_input[1:])

         x_input = x_input.reshape((1, n_steps , 1))
         yhat = model.predict(x_input,verbose = 0)

         temp_input.extend(yhat[0].tolist())
         temp_input = temp_input[1:]
         lst_output.extend(yhat.tolist())
         k=k+1
       else:
         x_input = x_input.reshape((1,n_steps,1))
         yhat = model.predict(x_input,verbose = 0)
         
         temp_input.extend(yhat[0].tolist())
         lst_output.extend(yhat.tolist())
         k = k +1
     lst_output = scaler.inverse_transform(lst_output) 
     return lst_output
     


In [ ]:
dfs = []
predicted_dataframe=[]
folder_path = '/kaggle/input/mine-the-model-2023/Upload-Dataset/TRAIN'
for filename in os.listdir(folder_path):
  stock_df = os.path.join(folder_path,filename)
  predicted_dataframe =lstm_model1(stock_df)
  dfs.append(predicted_dataframe)
x = []
z = [0] * 5000
m = []
for filename in os.listdir(folder_path):
    a = filename[:3] + '_#' 
    for i in range(100):
        y = a + str(i+1)
        x.append(y)
for i in range(50):
    for k in range(100):
        u = 100*i + k
        z[u]  = float(dfs[i][k])

In [ ]:
cc = pd.DataFrame({'ID' : x , 'Price' : z})
cc.to_csv('SubmissionVC.csv' ,index = False)
cc